In [4]:
# !pip install yfinance
import alpaca_trade_api as tradeapi
import os    
from dotenv  import load_dotenv
from pathlib import Path
import pandas as pd
import yfinance as yf

## Yahoo Finance

In [29]:
yf_stocks_df = yf.download(tickers = "DOW AAPL TRV BA JNJ PFE BTC-USD", interval='15m', start="2020-12-30", end="2021-02-15")
yf_stocks_df.head()

[*********************100%***********************]  7 of 7 completed


Adj Close                                   Close  \
                               AAPL  BA       BTC-USD DOW JNJ PFE TRV  AAPL   
Datetime                                                                      
2020-12-29 13:00:00+00:00       NaN NaN  26907.812500 NaN NaN NaN NaN   NaN   
2020-12-29 13:15:00+00:00       NaN NaN  26886.263672 NaN NaN NaN NaN   NaN   
2020-12-29 13:30:00+00:00       NaN NaN  26864.822266 NaN NaN NaN NaN   NaN   
2020-12-29 13:45:00+00:00       NaN NaN  26875.275391 NaN NaN NaN NaN   NaN   
2020-12-29 14:00:00+00:00       NaN NaN  26697.210938 NaN NaN NaN NaN   NaN   

                                             ... Open         Volume      \
                           BA       BTC-USD  ...  JNJ PFE TRV   AAPL  BA   
Datetime                                     ...                           
2020-12-29 13:00:00+00:00 NaN  26907.812500  ...  NaN NaN NaN    NaN NaN   
2020-12-29 13:15:00+00:00 NaN  26886.263672  ...  NaN NaN NaN    NaN NaN   
2020-12-29 13:30:00+00:00 NaN  26864.822266  ...  NaN NaN NaN    NaN NaN   
2020-12-29 13:45:00+00:00 NaN  26875.275391  ...  NaN NaN NaN    NaN NaN   
2020-12-29 14:00:00+00:00 NaN  26697.210938  ...  NaN NaN NaN    NaN NaN   

                                                       
                              BTC-USD DOW JNJ PFE TRV  
Datetime                                               
2020-12-29 13:00:00+00:00   183529472 NaN NaN NaN NaN  
2020-12-29 13:15:00+00:00           0 NaN NaN NaN NaN  
2020-12-29 13:30:00+00:00    38830080 NaN NaN NaN NaN  
2020-12-29 13:45:00+00:00  1509879808 NaN NaN NaN NaN  
2020-12-29 14:00:00+00:00   387457024 NaN NaN NaN NaN  

[5 rows x 42 columns]

## Alpaca API

In [10]:
# Load .env environment variables
load_dotenv()
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [14]:
# Format current date as ISO format
start = pd.Timestamp("2020-08-15", tz="America/New_York").isoformat()
end = pd.Timestamp("2021-02-15", tz="America/New_York").isoformat()
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "15Min"
# Set the tickers
tickers = ['DOW', 'AAPL', 'TRV', 'BA', 'JNJ', 'PFE']

In [30]:
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = start,
    end = end
).df
df_portfolio.head()

AAPL                             BA                  \
                          open high low close volume     open    high     low   
time                                                                            
2021-02-09 10:30:00-05:00  NaN  NaN NaN   NaN    NaN      NaN     NaN     NaN   
2021-02-09 10:45:00-05:00  NaN  NaN NaN   NaN    NaN      NaN     NaN     NaN   
2021-02-09 11:00:00-05:00  NaN  NaN NaN   NaN    NaN  211.980  212.33  211.69   
2021-02-09 11:15:00-05:00  NaN  NaN NaN   NaN    NaN  211.930  212.49  211.70   
2021-02-09 11:30:00-05:00  NaN  NaN NaN   NaN    NaN  212.555  215.84  212.50   

                                            ...  PFE                        \
                            close   volume  ... open high low close volume   
time                                        ...                              
2021-02-09 10:30:00-05:00     NaN      NaN  ...  NaN  NaN NaN   NaN    NaN   
2021-02-09 10:45:00-05:00     NaN      NaN  ...  NaN  NaN NaN   NaN    NaN   
2021-02-09 11:00:00-05:00  212.08  12268.0  ...  NaN  NaN NaN   NaN    NaN   
2021-02-09 11:15:00-05:00  212.42  12579.0  ...  NaN  NaN NaN   NaN    NaN   
2021-02-09 11:30:00-05:00  215.41  31319.0  ...  NaN  NaN NaN   NaN    NaN   

                               TRV                                  
                              open    high     low   close  volume  
time                                                                
2021-02-09 10:30:00-05:00  146.000  146.02  145.83  146.02  1896.0  
2021-02-09 10:45:00-05:00  146.220  146.33  146.09  146.33   986.0  
2021-02-09 11:00:00-05:00  146.425  146.65  146.30  146.58  2298.0  
2021-02-09 11:15:00-05:00  146.400  146.40  146.23  146.23   471.0  
2021-02-09 11:30:00-05:00  146.230  146.47  146.21  146.47  1351.0  

[5 rows x 30 columns]

## Alpha Api

In [5]:
from alpha_vantage.timeseries import TimeSeries

In [6]:
load_dotenv()
alpha_api=os.getenv('ALPHA')
ts = TimeSeries(key='B61MWTGKUJIK29G1',output_format='csv')

In [17]:
def get_price(symbol, frq, interval):
    df = []
    data, meta= ts.get_intraday_extended(symbol=symbol, interval=frq, slice=interval)
    for row in data:
        df.append(row)
    df = pd.DataFrame(df, columns=df[0])
    df.drop(index=0,inplace=True)
    
    return df

In [21]:
def get_whole(symbol, frq):
    df1 = get_price(symbol, frq, 'year1month1')
    df2 = get_price(symbol, frq, 'year1month2')
    df3 = get_price(symbol, frq, 'year1month3')
    df4 = get_price(symbol, frq, 'year1month4')
    df5 = get_price(symbol, frq, 'year1month5')
    df6 = get_price(symbol, frq, 'year1month6')
    df7 = get_price(symbol, frq, 'year1month7')
    df_whole = pd.concat([df1,df2,df3,df4,df5,df6],  axis=0).set_index('time').sort_index()
    return df_whole

In [22]:
df_pfe = get_whole('PFE', '30min')


C:\Users\Shan\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [23]:
df_pfe

,close,high,low,open,volume,{
time,,,,,,
2020-10-05 04:30:00,33.9962625466,33.9962625466,33.7919134275,33.8197792165,1826,NaN
2020-10-05 05:30:00,34.0427055283,34.0427055283,34.0427055283,34.0427055283,422,NaN
2020-10-05 06:30:00,34.0705713172,34.0705713172,33.9962625466,33.9962625466,511,NaN
2020-10-05 07:00:00,34.0798599136,34.0798599136,34.0798599136,34.0798599136,350,NaN
2020-10-05 07:30:00,34.0427055283,34.0612827209,33.9962625466,33.9962625466,927,NaN
2020-10-05 08:00:00,34.0334169319,34.0334169319,34.0334169319,34.0334169319,188,NaN
2020-10-05 08:30:00,34.061189835,34.0798599136,33.949819565,34.0705713172,13076,NaN
2020-10-05 09:00:00,33.9962625466,34.061189835,33.9869739503,34.0519941246,6664,NaN
2020-10-05 09:30:00,33.9405309687,34.0426126423,33.9405309687,33.9869739503,22057,NaN


In [42]:
df_pfe.to_csv('Pfizer.csv')

In [67]:
ts = TimeSeries(key=alpha_api,output_format='csv')

In [68]:
df1_JNJ = get_price('JNJ','year1month7')

In [69]:
df2_JNJ = get_price('JNJ','year1month6')

In [70]:
df3_JNJ = get_price('JNJ','year1month5')

In [71]:
df4_JNJ = get_price('JNJ','year1month4')

In [72]:
df5_JNJ = get_price('JNJ','year1month3')

In [73]:
df6_JNJ = get_price('JNJ','year1month2')

In [74]:
df7_JNJ = get_price('JNJ','year1month1')

In [75]:
df_jnj= pd.concat([df1_JNJ,df2_JNJ,df3_JNJ,df4_JNJ,df5_JNJ,df6_JNJ,df7_JNJ],  axis=0, sort=True).set_index('time').sort_index()
df_jnj

KeyError: 'time'

In [ ]:
df_jnj.to_csv('Jhonson.csv')